In [ ]:
işe_alım_destek_sistemi/
│
├── data/
│   ├── resumes/          # Adayların özgeçmiş dosyaları
│   ├── job_descriptions/ # İş ilanları
│   └── processed/        # İşlenmiş veri dosyaları
│
├── models/
│   ├── nlp_models/       # NLP modelleri
│   └── ml_models/        # ML ve DL modelleri
│
├── src/
│   ├── data_preprocessing.py
│   ├── feature_extraction.py
│   ├── model_training.py
│   ├── resume_analysis.py
│   └── job_matching.py
│
├── notebooks/
│   ├── data_exploration.ipynb
│   ├── model_evaluation.ipynb
│   └── results_visualization.ipynb
│
├── app.py                # Ana uygulama dosyası
├── requirements.txt
└── README.md

import os
import pandas as pd
import spacy
from sklearn.model_selection import train_test_split

nlp = spacy.load("en_core_web_sm")

def load_data(resume_dir, job_dir):
    resumes = []
    for file in os.listdir(resume_dir):
        if file.endswith(".txt"):
            with open(os.path.join(resume_dir, file), 'r') as f:
                resumes.append(f.read())
    
    job_descriptions = []
    for file in os.listdir(job_dir):
        if file.endswith(".txt"):
            with open(os.path.join(job_dir, file), 'r') as f:
                job_descriptions.append(f.read())
    
    return resumes, job_descriptions

def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

def preprocess_data(resumes, job_descriptions):
    resumes = [preprocess_text(resume) for resume in resumes]
    job_descriptions = [preprocess_text(job) for job in job_descriptions]
    return resumes, job_descriptions

if __name__ == "__main__":
    resume_dir = "../data/resumes"
    job_dir = "../data/job_descriptions"
    
    resumes, job_descriptions = load_data(resume_dir, job_dir)
    resumes, job_descriptions = preprocess_data(resumes, job_descriptions)
    
    resume_df = pd.DataFrame({'text': resumes, 'type': 'resume'})
    job_df = pd.DataFrame({'text': job_descriptions, 'type': 'job_description'})
    
    df = pd.concat([resume_df, job_df], ignore_index=True)
    df.to_csv("../data/processed/data.csv", index=False)


import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

def extract_features(data_path):
    df = pd.read_csv(data_path)
    vectorizer = TfidfVectorizer(max_features=5000)
    X = vectorizer.fit_transform(df['text'])
    return X, df['type']

if __name__ == "__main__":
    data_path = "../data/processed/data.csv"
    X, y = extract_features(data_path)
    
    from scipy.sparse import save_npz
    save_npz("../data/processed/features.npz", X)
    y.to_csv("../data/processed/labels.csv", index=False)


    import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from scipy.sparse import load_npz

def train_model(features_path, labels_path):
    X = load_npz(features_path)
    y = pd.read_csv(labels_path)['type']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = SVC(kernel='linear')
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))
    
    return model

if __name__ == "__main__":
    features_path = "../data/processed/features.npz"
    labels_path = "../data/processed/labels.csv"
    model = train_model(features_path, labels_path)
    
    import joblib
    joblib.dump(model, "../models/ml_models/svm_model.joblib")


import spacy
import joblib

nlp = spacy.load("en_core_web_sm")
model = joblib.load("../models/ml_models/svm_model.joblib")

def analyze_resume(resume_text):
    doc = nlp(resume_text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

def predict_resume_category(preprocessed_text):
    return model.predict([preprocessed_text])[0]

if __name__ == "__main__":
    resume_text = "Your resume text here."
    preprocessed_text = analyze_resume(resume_text)
    category = predict_resume_category(preprocessed_text)
    print(f"Predicted Category: {category}")

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import load_npz

def match_jobs(resume_features, job_features_path):
    job_features = load_npz(job_features_path)
    similarities = cosine_similarity(resume_features, job_features)
    return similarities

if __name__ == "__main__":
    resume_features_path = "../data/processed/resume_features.npz"
    job_features_path = "../data/processed/job_features.npz"
    
    resume_features = load_npz(resume_features_path)
    similarities = match_jobs(resume_features, job_features_path)
    
    job_df = pd.read_csv("../data/processed/job_descriptions.csv")
    top_jobs = job_df.iloc[similarities[0].argsort()[-5:][::-1]]
    print(top_jobs)


from flask import Flask, request, jsonify
import joblib
from src.resume_analysis import analyze_resume, predict_resume_category
from src.job_matching import match_jobs
from scipy.sparse import load_npz

app = Flask(__name__)

@app.route('/analyze_resume', methods=['POST'])
def analyze_resume_route():
    data = request.get_json()
    resume_text = data['resume_text']
    preprocessed_text = analyze_resume(resume_text)
    category = predict_resume_category(preprocessed_text)
    return jsonify({'category': category})

@app.route('/match_jobs', methods=['POST'])
def match_jobs_route():
    data = request.get_json()
    resume_features_path = data['resume_features_path']
    job_features_path = "../data/processed/job_features.npz"
    
    resume_features = load_npz(resume_features_path)
    similarities = match_jobs(resume_features, job_features_path)
    
    job_df = pd.read_csv("../data/processed/job_descriptions.csv")
    top_jobs = job_df.iloc[similarities[0].argsort()[-5:][::-1]]
    return top_jobs.to_json(orient="records")

if __name__ == "__main__":
    app.run(debug=True)


